<a href="https://colab.research.google.com/github/mohdbilal14/Kaggle_Comp_hull-starter-notebook.ipynb/blob/main/Kaggle_Comp_hull_starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

# Adjust the path if your files are in a folder
train_path = "/content/drive/My Drive/Backup/Mohd Bilal Siddiqui/Personal /Competition/Kaggle/Hull Tachtical Market prediction/hull-tactical-market-prediction/train.csv"
test_path  = "/content/drive/My Drive/Backup/Mohd Bilal Siddiqui/Personal /Competition/Kaggle/Hull Tachtical Market prediction/hull-tactical-market-prediction/test.csv"

train = pd.read_csv(train_path)
test  = pd.read_csv(test_path)

print(train.shape, test.shape)
print(train.head())


(9021, 98) (10, 99)
   date_id  D1  D2  D3  D4  D5  D6  D7  D8  D9  ...  V3  V4  V5  V6  V7  V8  \
0        0   0   0   0   1   1   0   0   0   1  ... NaN NaN NaN NaN NaN NaN   
1        1   0   0   0   1   1   0   0   0   1  ... NaN NaN NaN NaN NaN NaN   
2        2   0   0   0   1   0   0   0   0   1  ... NaN NaN NaN NaN NaN NaN   
3        3   0   0   0   1   0   0   0   0   0  ... NaN NaN NaN NaN NaN NaN   
4        4   0   0   0   1   0   0   0   0   0  ... NaN NaN NaN NaN NaN NaN   

   V9  forward_returns  risk_free_rate  market_forward_excess_returns  
0 NaN        -0.002421        0.000301                      -0.003038  
1 NaN        -0.008495        0.000303                      -0.009114  
2 NaN        -0.009624        0.000301                      -0.010243  
3 NaN         0.004662        0.000299                       0.004046  
4 NaN        -0.011686        0.000299                      -0.012301  

[5 rows x 98 columns]


In [ ]:
!ls "/content/drive/My Drive/Backup/Mohd Bilal Siddiqui/Personal /Competition/Kaggle/Hull Tachtical Market prediction/hull-tactical-market-prediction/"

kaggle_evaluation  test.csv  train.csv


In [ ]:
 # 1. Setup
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
# 2. Quick Exploration
print(train.head())
print(train.info())
print(train.describe())

   date_id  D1  D2  D3  D4  D5  D6  D7  D8  D9  ...  V3  V4  V5  V6  V7  V8  \
0        0   0   0   0   1   1   0   0   0   1  ... NaN NaN NaN NaN NaN NaN   
1        1   0   0   0   1   1   0   0   0   1  ... NaN NaN NaN NaN NaN NaN   
2        2   0   0   0   1   0   0   0   0   1  ... NaN NaN NaN NaN NaN NaN   
3        3   0   0   0   1   0   0   0   0   0  ... NaN NaN NaN NaN NaN NaN   
4        4   0   0   0   1   0   0   0   0   0  ... NaN NaN NaN NaN NaN NaN   

   V9  forward_returns  risk_free_rate  market_forward_excess_returns  
0 NaN        -0.002421        0.000301                      -0.003038  
1 NaN        -0.008495        0.000303                      -0.009114  
2 NaN        -0.009624        0.000301                      -0.010243  
3 NaN         0.004662        0.000299                       0.004046  
4 NaN        -0.011686        0.000299                      -0.012301  

[5 rows x 98 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9021 entries, 0 to 9

In [ ]:
# Check missing values
missing = train.isnull().sum().sort_values(ascending=False)
print(missing.head(20))

E7     6969
V10    6049
S3     5733
M1     5547
M13    5540
M14    5540
M6     5043
V9     4539
S12    3537
M5     3283
M2     3217
S8     3009
M3     2018
E1     1784
P6     1638
E20    1616
P7     1616
P5     1574
V5     1512
V13    1511
dtype: int64


In [ ]:
missing_pct = train.isnull().mean().sort_values(ascending=False)
print(missing_pct.head(20))


E7     0.772531
V10    0.670547
S3     0.635517
M1     0.614899
M13    0.614123
M14    0.614123
M6     0.559029
V9     0.503159
S12    0.392085
M5     0.363929
M2     0.356612
S8     0.333555
M3     0.223700
E1     0.197761
P6     0.181576
E20    0.179138
P7     0.179138
P5     0.174482
V5     0.167609
V13    0.167498
dtype: float64


In [ ]:
#Baseline Imputation
train_filled = train.fillna(0)
test_filled  = test.fillna(0)


In [ ]:
# Threshold for dropping columns (70% missing)
threshold = 0.7

# Calculate missing percentage
missing_pct = train.isnull().mean()

# Identify columns to drop
drop_cols = missing_pct[missing_pct > threshold].index.tolist()
print("Dropping columns:", drop_cols)

# Drop high-missing columns
train_reduced = train.drop(columns=drop_cols)
test_reduced  = test.drop(columns=drop_cols)

# Median imputation for remaining columns
train_filled = train_reduced.fillna(train_reduced.median())
test_filled  = test_reduced.fillna(train_reduced.median())  # use train medians for consistency

print("Train shape after cleaning:", train_filled.shape)
print("Test shape after cleaning:", test_filled.shape)


Dropping columns: ['E7']
Train shape after cleaning: (9021, 97)
Test shape after cleaning: (10, 98)


In [ ]:
# Training features (from train.csv)
train_features = ["risk_free_rate", "market_forward_excess_returns"]
target = "forward_returns"

In [ ]:
X_train = train_filled[train_features]
y_train = train_filled[target]

In [ ]:
# Test features (lagged versions in test.csv)
test_features = ["lagged_risk_free_rate", "lagged_market_forward_excess_returns"]
X_test = test_filled[test_features]

In [ ]:
# Train model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
X_test = test_filled[["lagged_risk_free_rate", "lagged_market_forward_excess_returns"]].copy()
X_test.rename(columns={
    "lagged_risk_free_rate": "risk_free_rate",
    "lagged_market_forward_excess_returns": "market_forward_excess_returns"
}, inplace=True)

test_filled["prediction"] = model.predict(X_test)


In [ ]:
# Predict on test
test_filled["prediction"] = model.predict(X_test)

In [ ]:
# Evaluate correlation on scored rows
scored = test_filled[test_filled["is_scored"] == True]
corr = np.corrcoef(scored["prediction"], scored["lagged_forward_returns"])[0,1]
print("Correlation:", corr)

Correlation: 0.9999999961263665


In [ ]:
# Submission file
submission = test_filled[["date_id", "prediction"]]
submission.to_csv("submission.csv", index=False)